<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/end_to_end(modelA%EF%BC%8BB%EF%BC%8BC)tune_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install autorch
#!pip install ray[tune]
import pandas as pd
import numpy as np
import joblib
from torch import nn
from tqdm import tqdm_notebook as tqdm
import random
import autorch
from autorch.function import sp2wt
import torch
from ray import tune

In [2]:
c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/phase2/dataset/col_names.pkl')
c.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/phase2/dataset/train.csv',index_col=0)
df = df.dropna(axis=0).astype('float32')
df.head(3)

,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,...,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Component to Heavy End Split Factor_n-Heptane_Ratio,Individual Component to Heavy End Split Factor_H2O_Ratio,Individual Component to Heavy End Split Factor_Methylcyclohexane_Ratio,Individual Component to Heavy End Split Factor_Ethylcyclopentane_Ratio,"Individual Component to Heavy End Split Factor_2,3-Dimethyl-1-hexene_Ratio",Individual Component to Heavy End Split Factor_Toluene_Ratio,Individual Component to Heavy End Split Factor_2-Methylheptane_Ratio,Individual Component to Heavy End Split Factor_n-Octane_Ratio,Individual Component to Heavy End Split Factor_Ethylcyclohexane_Ratio,Individual Component to Heavy End Split Factor_EthylBenzene_Ratio,Individual Component to Heavy End Split Factor_P-Xylene_Ratio,Individual Component to Heavy End Split Factor_M-Xylene_Ratio,Individual Component to Heavy End Split Factor_O-Xylene_Ratio,Individual Component to Heavy End Split Factor_2-Methyloctane_Ratio,Individual Component to Heavy End Split Factor_1-Nonene_Ratio,Individual Component to Heavy End Split Factor_n-Nonane_Ratio,Individual Component to Heavy End Split Factor_i-Propylbenzene_Ratio,"Individual Component to Heavy End Split Factor_1-trans-3,5-Trimethylcyclohexane_Ratio",Individual Component to Heavy End Split Factor_n-Propylcyclohexane_Ratio,Individual Component to Heavy End Split Factor_n-Propylbenzene_Ratio,"Individual Component to Heavy End Split Factor_1,3,5-Trimethylbenzene_Ratio",Individual Component to Heavy End Split Factor_1-Methyl-2-ethylbenzene_Ratio,"Individual Component to Heavy End Split Factor_1,2,3-Trimethylbenzene_Ratio","Individual Component to Heavy End Split Factor_2,4-Dimethyloctane_Ratio","Individual Component to Heavy End Split Factor_2,2,3,3-Tetramethylhexane_Ratio",Individual Component to Heavy End Split Factor_3-Methylnonane_Ratio,Individual Component to Heavy End Split Factor_tert-Butylcyclohexane_Ratio,Individual Component to Heavy End Split Factor_n-Decane_Ratio,"Individual Component to Heavy End Split Factor_1,2-Diethylbenzene_Ratio",Individual Component to Heavy End Split Factor_n-Undecane_Ratio,Individual Component to Heavy End Split Factor_n-Pentylbenzene_Ratio,Individual Component to Heavy End Split Factor_n-Dodecane_Ratio,Individual Component to Heavy End Split Factor_n-Hexylbenzene_Ratio,Individual Component to Heavy End Split Factor_n-Tridecane_Ratio,Case Conditions_C7+ in Light End 

In [5]:
# X
X = df[c['case']+c['xna']]
print('X.shape',X.shape)

# modelA的Y
Y1 = df[c['sle']+c['shc']+c['she']]
print('Y1.shape',Y1.shape)

duty = df[c['duty']]
density = pd.DataFrame(df[c['Rate_ton']].values/df[c['Rate_m3']].values,
                       columns=['NA_density','LE_density','HC_density','HE_density'],
                       index=df.index)
# modelB的Y
Y2 = duty.join(density)
print('Y2.shape',Y2.shape)

# modelC的Y
Y3 = df[c['Tray_temp']+c['fna']+c['yld']]
print('Y3.shape',Y3.shape)

# (modelA + modelB + modelC)的Y
Y = Y1.join(Y2).join(Y3)
print('Y.shape',Y.shape)

X.shape (16498, 58)
Y1.shape (16498, 162)
Y2.shape (16498, 7)
Y3.shape (16498, 8)
Y.shape (16498, 177)


In [6]:
wt = df[c['xle']+c['xhc']+c['xhe']]
df = X.join(Y).join(wt)
x_col = X.columns.tolist()
y_col = Y.columns.tolist()

In [7]:
normalize_idx_list = []
for i in range(54):
  normalize_idx_list.append([y_col.index(c['sle'][i]),
                             y_col.index(c['shc'][i]),
                             y_col.index(c['she'][i])])
for idx in tqdm(normalize_idx_list):
  assert np.allclose(df[y_col].iloc[:,idx].sum(axis=1),1) # 因為分離係數要確保質量平衡
print('pass')


pass


In [8]:
def train_and_test(config):
  
  # 1. setting seed
  np.random.seed(config["seed"])
  random.seed(config["seed"])
  torch.manual_seed(config["seed"])
  torch.cuda.manual_seed(config["seed"])

  # 2. split ratio massblance
  normalize_idx_list = []
  for i in range(54):
    normalize_idx_list.append([y_col.index(c['sle'][i]),y_col.index(c['shc'][i]),y_col.index(c['she'][i])])
  
  # 3. model train
  model = autorch.utils.PartBulider(df,x_col,y_col,normalize_idx_list=normalize_idx_list,device='cuda:0') # 使用 gpu train 快一些
  model.train()

  # 4. model predict
  x_test = model.data['X_test']
  sp_pred,op_pred = model.predict(x_test).iloc[:,:162],model.predict(x_test).iloc[:,162:] # predict sp op

  # 5. sp2wt
  xna = x_test[c['xna']]
  xle = sp2wt(xna.values,sp_pred[c['sle']].values)
  xhc = sp2wt(xna.values,sp_pred[c['shc']].values)
  xhe = sp2wt(xna.values,sp_pred[c['she']].values)
  wt_pred = np.hstack((xle,xhc,xhe))
  wt_pred = pd.DataFrame(wt_pred,index=x_test.index,columns=c['xle']+c['xhc']+c['xhe'])

  # 6. wt ground truth
  wt_real = df.loc[x_test.index,c['xle']+c['xhc']+c['xhe']]
  
  # 7. result
  result = model.show_metrics(wt_real,wt_pred,e=0.02)

  # 8. tune report
  tune.report(r2 = result.loc['AVG','R2'],mse =result.loc['AVG','MSE'] ,mape=result.loc['AVG','MAPE'])

In [9]:
analysis = tune.run(
    train_and_test,
    resources_per_trial={'cpu':2 ,'gpu': 1},
    metric = 'mape',
    mode = 'min',
    verbose  = 2,
    config={"seed":tune.grid_search([*range(10000)])},
    )
print("Best config: ", analysis.get_best_config(metric='mape'))

2020-12-12 04:03:07,855	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-12 04:03:10,652	WARNING function_runner.py:540 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2020-12-12 04:03:11,645	WARNING worker.py:1091 -- Warning: The actor ImplicitFunc has size 26399882 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,seed
train_and_test_f30e2_00000,RUNNING,,0


(pid=2218)   Running command git clone -q https://github.com/jonbarron/robust_loss_pytorch /tmp/pip-req-build-c0x8smtd


(pid=2218) Collecting git+https://github.com/jonbarron/robust_loss_pytorch
(pid=2218)   Cloning https://github.com/jonbarron/robust_loss_pytorch to /tmp/pip-req-build-c0x8smtd
(pid=2218) Requirement already satisfied (use --upgrade to upgrade): robust-loss-pytorch==0.0.2 from git+https://github.com/jonbarron/robust_loss_pytorch in /usr/local/lib/python3.6/dist-packages
(pid=2218) Requirement already satisfied: torch>=1.3.1 in /usr/local/lib/python3.6/dist-packages (from robust-loss-pytorch==0.0.2) (1.7.0+cu101)
(pid=2218) Requirement already satisfied: torch-dct in /usr/local/lib/python3.6/dist-packages (from robust-loss-pytorch==0.0.2) (0.1.5)
(pid=2218) Requirement already satisfied: numpy>=1.15.4 in /usr/local/lib/python3.6/dist-packages (from robust-loss-pytorch==0.0.2) (1.18.5)
(pid=2218) Requirement already satisfied: scipy>=1.1.0 in /usr/local/lib/python3.6/dist-packages (from robust-loss-pytorch==0.0.2) (1.4.1)
(pid=2218) Requirement already satisfied: absl-py>=0.1.9 in /usr/lo

  0%|          | 0/300 [00:00<?, ?it/s]


...

(pid=4991) epoch:50 train_loss:0.0002 valid_loss:0.0002


 33%|███▎      | 100/300 [01:14<02:29,  1.34it/s]


(pid=4991) epoch:100 train_loss:0.0001 valid_loss:0.0002


 34%|███▍      | 103/300 [01:16<02:31,  1.30it/s]


KeyboardInterrupt: ignored